## Scrape detailed information about an athlete

This file takes the list of athletes found in scrape_athlete_list.ipynb and pulls detailed information about their events.

Each athlete comes in with the following information:
* Name
* ID
* School
* School ID
* Conference

After processing in this file the additional informacion is added to their record
* Freshman PR
* Sophomore PR
* Junior PR
* Senior PR
* Year they used last year of eligibility

In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os






### Pull data for several athletes

Loop over a list of athletes from a csv file and read their 100 PR's

In [39]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [40]:
# given a time string, return a float for the time in seconds
def cleantime(time_str):
    time_str = time_str.replace('PR','')
    time_str = time_str.replace('*','')
    time_str = time_str.replace('c','')
    time_str = time_str.replace('h','')
    time_str = time_str.replace('m', '')

    time_str = time_str.strip()
    if "'" in time_str:
        t = time_str.split("'")
        ft = float(t[0])
        inch = float(t[1]) / 12
        fin = ft + inch
        return fin
    
    #print('time = ',time_str)
    if ':' in time_str:
        t = time_str.split(':')
        time = float(t[0])*60 + float(t[1])
    elif time_str.replace('.','').isdigit(): 
        time = float(time_str)
    else: time = np.nan
    #print(time)
    return(time)

In [41]:
os.getcwd()

'/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/code'

In [42]:
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athletes.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_boy1600.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_girlls 1600.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_girl400.csv'
file = '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_womenPV.csv'
#file = '/Users/dana/metis/git/Metis_Bootcamp/Luther_Project/athlete_list_boy400.csv'

athlete_list = pd.read_csv(file)

In [43]:
athlete_list.head()

,ID,Name,School_ID,School,Division
0,12540940,Mallory Prelewicz,20661,Binghamton,America East
1,14308652,Samantha Martinez,20591,Albany,America East
2,12540793,Jordan Hoffman,21176,New Hampshire,America East
3,16204866,Taylor Nydam,21176,New Hampshire,America East
4,17659311,Sarah Morin,21176,New Hampshire,America East


In [44]:
# Given a list of athletes, scrap their PR times in the 400Meter race
# and add to dataframe
all_data = athlete_list[['ID','Name','School_ID','School','Division']]
all_data = all_data.set_index(['ID'])
all_data['Fr_PR']  = np.nan
all_data['So_PR'] = np.nan
all_data['Jr_PR'] = np.nan
all_data['Sr_PR'] = np.nan
all_data['Grad_Yr'] = 0
all_data.Grad_Yr = all_data.Grad_Yr.astype(int)

In [45]:
all_data.head()

,Name,School_ID,School,Division,Fr_PR,So_PR,Jr_PR,Sr_PR,Grad_Yr
ID,,,,,,,,,
12540940,Mallory Prelewicz,20661,Binghamton,America East,NaN,NaN,NaN,NaN,0
14308652,Samantha Martinez,20591,Albany,America East,NaN,NaN,NaN,NaN,0
12540793,Jordan Hoffman,21176,New Hampshire,America East,NaN,NaN,NaN,NaN,0
16204866,Taylor Nydam,21176,New Hampshire,America East,NaN,NaN,NaN,NaN,0
17659311,Sarah Morin,21176,New Hampshire,America East,NaN,NaN,NaN,NaN,0


In [46]:
all_data.shape

(5666, 9)

In [47]:
all_data_test = all_data[:10].copy()

In [53]:
files = [#'/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_women100.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_men100.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_women5000.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_men5000.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_womenPV.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_menPV.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_womenTJ.csv',
        '/Users/etalius/Downloads/Metis_Bootcamp-master/2-Luther_Project/athlete_list_menTJ.csv']
events = [ "100 Meters", "5000 Meters", "5000 Meters", 
          "Pole Vault", "Pole Vault", "Triple Jump", "Triple Jump"]

In [33]:
#all_data = all_data4

Event = 'Pole Vault'
#Event = '1600 Meters'
n = 0
for ID, row in all_data_test.iterrows():
    ID_str = str(ID)
    url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+ID_str+'#!/L0'
    r = requests.get(url, headers=header)
    dfs = pd.read_html(r.text)
    for df in dfs:
        #print(df)
        if df[0][0] == Event:
            n += 1
            if n%100 == 0: print(n)

            df.drop(df.index[0:1],inplace=True)
            for index, row in df.iterrows():
                if 'Indoor' not in str(row[0]):
                    #print(f"grade {int(row[1])}, time {row[2]}")
                    #row[1]= str(row[1]).replace('-','')
                    g = str(row[1])
                    g = g.replace('-','')
                    g = g.replace('.0','')

                    #print(ID_str,g)
                    if (g == 'Fr' or g == 'So' or g == 'Jr' or g == 'Sr'): 
                        #grade = str(int(row[1]))
                        grade = g

                    else:
                        grade = '0'
                    time = cleantime(str(row[2]))
                    #print(grade,time)
                    all_data_test.loc[ID,grade+"_PR"] = time
                    #print
# find the graduation year
                    if g == 'Sr': 
                        gy = str(row[0])
                        gy = gy.replace(' Outdoor','')
                        all_data_test.loc[ID,'Grad_Yr'] = int(gy)
                        #print(int(gy))


In [ ]:
for i in range(len(files)):
    file = files[i]
    Event = events[i]
    print(Event)
    
    all_data = athlete_list = pd.read_csv(file)
    all_data = athlete_list[['ID','Name','School_ID','School','Division']]
    all_data = all_data.set_index(['ID'])
    all_data['Fr_PR']  = np.nan
    all_data['So_PR'] = np.nan
    all_data['Jr_PR'] = np.nan
    all_data['Sr_PR'] = np.nan
    all_data['Grad_Yr'] = 0
    all_data.Grad_Yr = all_data.Grad_Yr.astype(int)
    
    save_path = Event + " " + str(i) + "results.csv"
    
    print(all_data.shape)


    n = 0
    for ID, row in all_data.iterrows():
        ID_str = str(ID)
        url = 'https://www.athletic.net/TrackAndField/Athlete.aspx?AID='+ID_str+'#!/L0'
        r = requests.get(url, headers=header)
        dfs = pd.read_html(r.text)
        for df in dfs:

            if df[0][0] == Event:
                n += 1
                if n%100 == 0: print(n)

                df.drop(df.index[0:1],inplace=True)
                for index, row in df.iterrows():
                    if 'Indoor' not in str(row[0]):
                        g = str(row[1])
                        g = g.replace('-','')
                        g = g.replace('.0','')


                        if (g == 'Fr' or g == 'So' or g == 'Jr' or g == 'Sr'): 
                            grade = g
                        else:
                            grade = '0'
                        time = cleantime(str(row[2]))

                        all_data.loc[ID,grade+"_PR"] = time

                        if g == 'Sr': 
                            gy = str(row[0])
                            gy = gy.replace(' Outdoor','')
                            all_data.loc[ID,'Grad_Yr'] = int(gy)
    
    all_data.drop(['0_PR'], axis=1,inplace=True)
    all_data.dropna(inplace=True)
    print(all_data.shape)
    all_data.to_csv(save_path)
    
    

100 Meters
(13359, 9)
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
(1523, 9)
5000 Meters
(14536, 9)
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300

In [39]:
all_data.drop(['0_PR'], axis=1,inplace=True)

In [36]:
all_data

,Name,School_ID,School,Division,Fr_PR,So_PR,Jr_PR,Sr_PR,Grad_Yr,0_PR
ID,,,,,,,,,,
5029205,Samiyah Samuels,20955,Houston,0,11.74,11.72,11.58,12.17,2021,11.80
9122821,Teahna Daniels,21452,Texas,0,11.21,11.06,11.11,10.99,2019,12.01
12552934,MaryBeth Sant,20768,Colorado State,0,11.47,11.57,11.71,11.41,2019,11.26
9125767,Kortnei Johnson,21056,LSU,0,11.27,11.09,10.97,11.47,2019,11.26
7157210,Tyriete Dixon,21462,Texas-Arlington,0,12.28,11.81,11.87,12.06,2019,12.44
8085366,Leya Buchanan,21232,Oklahoma,0,11.68,11.49,11.15,11.17,2018,11.56
7715666,Persis William-Mensah,21486,UC Irvine,0,11.72,11.55,11.41,11.42,2018,11.61
7260152,Ashley Henderson,21326,San Diego State,0,11.64,10.96,11.01,10.91,2018,11.41
7694201,Destiny Smith-Barnett,21174,Nevada-Las Vegas,0,11.67,11.32,10.94,11.25,2018,11.32


all_data1.shape